# Specifications:
1)
    - Embedding model: Huggingface(sentence-transformers/all-mpnet-base-v2)
    - LLM: mistralai/Mistral-7B-Instruct-v0.3
2)
    - Embedding model: OllamaEmbedding(nomic-embed-tex0t v1.5)
    - LLM: mistralai/Mistral-7B-Instruct-v0.3    

# 1) OllamaLLm and HuggingFaceEmbedding: mistralai/Mistral-7B-Instruct-v04

## Document Loader
    pdf loader : langchain inbuilt document loader 

In [2]:
from langchain_community.document_loaders import PyPDFLoader
file_path = (r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\3_Text_query_bot\_docs\Leave_Policy_2024.pdf")
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()
len(pages)


4

## Split
    smaller chunks

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
len(splits)


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

## Create vector store
    stores embeddings of Documents

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [ ]:
## Locally loading embedding
### Still pending --------------------------->>>>>>>>>>>>>>
# new_embeddings = HuggingFaceEmbeddings(cache_folder= r"C:\Users\30078206\Downloads\sentence-transformersall-mpnet-base-v2")


In [ ]:
## Not local
new_embeddings = HuggingFaceEmbeddings(model_name= "sentence-transformers/all-mpnet-base-v2")
vectorstore = Chroma.from_documents(documents=splits, embedding=new_embeddings)
vectorstore

## Create retriever

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity")

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [ ]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# !ollama pull mistral

In [ ]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [ ]:

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)


### New Terms:
- Multiqueryretriever: 
    - automates process of tuning
    - to generate multiple queries from different perspective 
    - for each query- returns relevant documents,, takes union across all 
    - Overcomes the limitation of distance based retrieval


In [ ]:
retriever = MultiQueryRetriever.from_llm(
    retriever, 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
questions = chain.invoke("Make a list of questions from the document")

In [ ]:
import re

In [ ]:
answer =re.sub(r'[0-9]','',questions)
question_list = answer.split('\n')
question_list

In [ ]:
chain.invoke(question_list[0])

In [ ]:
''' Taking too much time'''
# answer_list = []
# for question in question_list:
#     ans = chain.invoke(question)
#     answer_list.append(ans)

In [ ]:
print("ANSWERS-----")
# print(answer_list)

In [ ]:
question_bank = [
    "Give me a summary of the document",
    "Tell me all about the leaves",
    "Make a list of questions from the document"
]

--------------------------------------------(2)--------------------------------------------

# 2)OllamaLLM and OllamaEmbedding: mistralai/Mistral-7B-Instruct-v04

## Document Loader
    pdf loader : langchain inbuilt document loader 

In [3]:
from langchain_community.document_loaders import PyPDFLoader
file_path = (r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\3_Text_query_bot\_docs\Leave_Policy_2024.pdf")
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()
len(pages)


4

## Split
    smaller chunks

### Variables:
- types of splitters, embeddings, vector_store
- chunk_size:
- chunk_overlap:
    

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
len(splits)


15

In [3]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma

## Create vector store
    stores embeddings of Documents

### New terms:
- show progres
- collection name

In [4]:
embeddings = OllamaEmbeddings(model= "nomic-embed-text", show_progress=True)
vectorstore = Chroma.from_documents(documents=splits, embedding= embeddings)
vectorstore

OllamaEmbeddings: 100%|██████████| 15/15 [00:46<00:00,  3.12s/it]


## Create retriever

In [5]:
retriever = vectorstore.as_retriever(search_type="similarity")

In [6]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [7]:
# !ollama pull mistral

In [8]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [9]:

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)


### New Terms:
- Multiqueryretriever: 
    - automates process of tuning
    - to generate multiple queries from different perspective 
    - for each query- returns relevant documents,, takes union across all 
    - Overcomes the limitation of distance based retrieval


In [10]:
retriever = MultiQueryRetriever.from_llm(
    retriever, 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [11]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [12]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
questions = chain.invoke("Make a list of questions from the document")

INFO:langchain.retrievers.multi_query:Generated queries: ['1. List down the questions presented in the document.', '2. Can you provide me with the questions found within the given document?', '3. What are the queries contained in the specified document?', '4. I need a list of the questions that appear in the provided document, could you help with that?', "5. Could you retrieve a list of questions from the document I've mentioned?"]
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.23s/it]


KeyboardInterrupt: 

In [ ]:
import re

In [ ]:
answer =re.sub(r'[0-9]','',questions)
question_list = answer.split('\n')
question_list

In [ ]:
answer = chain.invoke(question_list[0])

In [ ]:
print(f"Question: {question_list[0]}")
print(f"Answer:{answer}")

### Conclusions:
- Using OllamaEmbeddings generated more questions when asked

